In this kata your task is to differentiate a mathematical expression given as a string in prefix notation. The result should be the derivative of the expression returned in prefix notation.

To simplify things we will use a simple list format made up of parentesis and spaces.

    The expression format is (func arg1) or (op arg1 arg2) where op means operator, func means function and arg1, arg2 are aguments to the operator or function. For example (+ x 1) or (cos x)

    The expressions will always have balanced parentesis and with spaces between list items.

    Expression operators, functions and arguments will all be lowercase.

    Expressions are single variable expressions using x as the variable.

    Expressions can have nested arguments at any depth for example (+ (* 1 x) (* 2 (+ x 1)))

Examples of prefix notation in this format:

(+ x 2)        // prefix notation version of x+2

(* (+ x 3) 5)  // same as 5 * (x + 3)

(cos (+ x 1))  // same as cos(x+1)

(^ x 2)        // same as x^2 meaning x raised to power of 2

The operators and functions you are required to implement are + - * / ^ cos sin tan exp ln where ^ means raised to power of. exp is the exponential function (same as e^x) and ln is the natural logarithm (base e).

Example of input values and their derivatives:

(* 1 x) => 1

(^ x 3) => (* 3 (^ x 2))

(cos x) => (* -1 (sin x))

In addition to returning the derivative your solution must also do some simplifications of the result but only what is specified below.

    The returned expression should not have unecessary 0 or 1 factors. For example it should not return (* 1 (+ x 1)) but simply the term (+ x 1) similarly it should not return (* 0 (+ x 1)) instead it should return just 0

    Results with two constant values such as for example (+ 2 2) should be evaluated and returned as a single value 4

    Any argument raised to the zero power should return 1 and if raised to 1 should return the same value or variable. For example (^ x 0) should return 1 and (^ x 1) should return x

    No simplifactions are expected for functions like cos, sin, exp, ln... (but their arguments might require a simplification).

Think recursively and build your answer according to the rules of derivation and sample test cases.

# NOTE that this is not an accepted answer as CodeWars does not allow sympy to be imported.  However, it was a useful exercise in converting between prefix notation and Python notation!

I have not attempted any work on simplication of the entries, nor has my testing been exhaustive

In [340]:
# join entries until the number of opening and closing brackets matches
def join_entries(entries, i):
    count_start_bracket = count_end_bracket = 0
    arg = entry = entries[i]

    # count the opening and closing brackets in the current entry
    count_start_bracket += entry.count('(')
    count_end_bracket += entry.count(')')
    
    while count_end_bracket < count_start_bracket and i < len(entries) - 1:
        # increment the count with brackets in the next entry
        count_start_bracket += entries[i+1].count('(')
        count_end_bracket += entries[i+1].count(')')
                   
        # join this entry with the next and delete the merged entry
        entries[i] = entries[i] + ' ' + entries[i+1]
        del entries[i+1]
        entry = entries[i]
    # convert the merged entry from prefix notation into Python notation
    arg = from_prefix_notation(entry)
    return entries, arg

In [341]:
def from_prefix_notation(s):
    # if first character is ( and final character is ), remove both
    if s[0] == '(' and s[-1] == ')':
        s = s[1:-1]

    # split argument at white space - if there's only one entry, no processing is needed
    entries = s.split()
    if len(entries) == 1:
        return s

    # process the entries to determine operator and arguments
    arg = function = ''
    i = 0
    while i < len(entries):
        entry = entries[i]
        # if there's an opening bracket and it's not the first character, there must be a function
        if entry.find('(') not in [-1, 0]:
            function = entry[:entry.find('(')]
            entry = entry[entry.find('('):]

        # determine operator
        if i % 3 == 0:
            if entry[0] == '(':
                operator = entry[1:]
            else:
                operator = entry
            if operator == '^':
                operator = '**'

        # determine first argument - if it contains brackets, need to combine with the following
        # entry/entries until the number of end brackets matches the number of start brackets
        elif i % 3 == 1:
            arg1 = entry
            if entry.count('(') > 0:
                entries, arg1 = join_entries(entries, i)
            
        # determine second argument - if it contains brackets, need to combine with the following
        # entry/entries until the number of end brackets matches the number of start brackets
        else:
            arg2 = entry
            if entry.count('(') > 0:
                entries, arg2 = join_entries(entries, i)

            # construct argument
            arg = function + '(' + arg1 + ' ' + operator + ' ' + arg2
            if len(function) == 0:
                arg += ')'

        # increment counter
        i += 1

    return arg

In [342]:
def process_operator(s, operator):
    arg = s
    converted_arg = ''
    pos = arg.find(operator)
    if pos == -1:
        return s

    while pos != -1:
        # work backwards from start of operator until finding another operator or the start of the string.  
        # If an end bracket is found, we need to find the matching start bracket
        substr = arg[:pos]
        arg1 = substr
        before = ''
        function = ''
        count_start_bracket = count_end_bracket = 0
        for i in range(len(substr)-1, -1, -1):
            if substr[i] in '*/+-' and count_end_bracket == 0:
                arg1 = substr[i+1]
                before = substr[:i+1]
                break
            elif substr[i] == ')':
                count_end_bracket += 1
            elif substr[i] == '(':
                if ((count_start_bracket == count_end_bracket) or (count_start_bracket + 1 == count_end_bracket)) and (i>2 and substr[i-3:i] in ['sin', 'cos', 'tan', 'exp']) or (i>1 and substr[i-2:i] == 'ln'):
                    function = 'ln'
                    if substr[i-2:i] != function:
                        function = substr[i-3:i]

                    # if we haven't found an end bracket then the operator is inside the function call
                    if count_end_bracket == 0:
                        arg1 = substr[i+1:]
                        before = substr[:i-len(function)]

                    # otherwise the operator is after the function call
                    else:
                        arg1 = substr[i-len(function):]
                        before = substr[:i-len(function)]
                        function = ''
                    break
                else:
                    function = ''
                    count_start_bracket += 1
                    if count_start_bracket == count_end_bracket:
                        arg1 = substr[i:]
                        before = substr[:1]
                        count_start_bracket = count_end_bracket = 0
                    
        # work forwards from end of operator until finding another operator or the end of the string.  
        # If a start bracket is found, we need to find the matching end bracket
        substr = arg[pos+len(operator):]
        arg2 = substr
        after = ''
        count_start_bracket = count_end_bracket = 0
        for i in range(len(substr)):
            if substr[i] in '*/+-':
                arg2 = substr[:i]
                after = substr[i:]
                break
            elif substr[i] == '(':
                count_start_bracket += 1
            elif substr[i] == ')':
                count_end_bracket += 1
                if count_end_bracket == count_start_bracket:
                    arg2 = substr[:i+1]
                    after = substr[i+1:]
                    count_start_bracket = count_end_bracket = 0

        # reconstruct this part of the argument using prefix notation
        if operator == '**':
            operator = '^'
        if len(arg1) == 0:
            arg1 = converted_arg
            converted_arg = ''
        converted_arg += before + ' ' + function + '(' + operator + ' ' + arg1 + ' ' + arg2 + ')'

        # check if there's another instance of the operator
        arg = after
        pos = arg.find(operator)

    # once no more instances of the operator found, append the remaining text to the string
    converted_arg += arg

    # return the string in prefix notation
    return converted_arg

In [346]:
def to_prefix_notation(s):

    # operator with highest priority is **
    s = process_operator(s, '**')

    # operators with next highest priority are * and /
    s = process_operator(s, '*')
    s = process_operator(s, '/')

    # operators with lowest priority are + and -
    s = process_operator(s, '+')
    s = process_operator(s, '-')
    
    return s

In [344]:
def diff(s):
    # convert prefix notation into python notation
    converted = from_prefix_notation(s)

    # differentiate the expression
    import sympy as sym
    result = str(sym.diff(converted))

    # convert result back into prefix notation
    return to_prefix_notation(result).strip().replace('  ', ' ')

In [347]:
diff( 'exp(x)' )

'exp(x)'

In [348]:
diff( 'ln(x)')

'(/ 1 x)'

In [349]:
diff('(* 2 (^ x 5))')

'(* 10 (^ x 4))'

In [350]:
diff('sin(^ x 2)')

'(* (* 2 x) cos(^ x 2)))'

In [351]:
diff('tan(x)')

'(+ (^ tan(x) 2 ) 1)'